In [1]:
import numpy as np  # http://www.numpy.org
import matplotlib.pyplot as plt   # http://matplotlib.org
import math as M  # https://docs.python.org/2/library/math.html
from sympy.solvers import solve
from sympy import Symbol
from scipy.optimize import fsolve

# Simple model including salinity

In [2]:
#### Constants #####
L_x = 1000000 #zonal extent [m]
L_y = 2000000 #meridional extent [m]
A = L_x*L_y #surface area of interior [m^2] Note that the interior is probably smaller than this
P = 2*L_x+2*L_y #Perimeter [m]

g = 9.81 #gravitational acceleration [m/s^2]
C_p = 3994 #specific heat capacity of sea water [J/(kg K)] source: Steffie's paper
alpha_T = 0.2 #thermal expansion coefficient [kg m^-3 C^-1] source: Steffie's paper
alpha_S = 0.8 #Haline expansion coefficient [kg m^-3]
c = 0.006 #bottom slope coefficient
s = 0.02 #average bottom slope around the perimeter of the marginal sea
S = 35 #reference salinity [ppt]. Spall [2015]
E = -2e-8 #constant freshwater flux [m/s]
S_1 = 35.2 #from Lambert [g kg^-1]

#### Parameters ####
H = 1000 #sill height [m]
f_0 = 1.2e-4 #Coriolis parameter [s^-1]
Gamma = 20 #Restoring strength [W/(m^2 C)]

L = H/s #width of sloping topography of the boundary current [m]
T_1 = 8.87 #ingoing temperature [Celsius]
T_A = 2.8 #Atmospheric temperature [Celsius]
ro_0 = 1028  #density sea water [kg m^-3]. Range of 7 to 9 degrees of sea water
#h_1 = ... #thickness upper layer [m]

#### functions ####
epsilon = c*P/L #ratio of heat fluxed into interior by eddies compared to heat flux by advection into the basin in the inflowing boundary current [-]
mu = A*Gamma*f_0/(alpha_T*g*C_p*H**2*(T_1-T_A)) #[-]
gamma = 8*A*ro_0*f_0*S*alpha_S*E/(g*H**2*alpha_T**2*(T_1-T_A)**2)


In [3]:
############# Computation ##################
############################################

# Main equations including salinity in the equation of state

def main(x): # function with seven unkowns:  T_0,  T_2,  S_0,  S_2,  V_1, T_eddy, S_eddy 
             #                              x[0], x[1], x[2], x[3], x[4],   x[5],   x[6]
    return [P*H*x[5]-A*Gamma*(x[0]-T_A)/(ro_0*C_p),
            P*H*x[6]+A*E*S,
            (T_1-x[1])*x[4]*H*L-(x[0] - T_A + (P*L/A)*(T_1-T_A))*A*Gamma/(ro_0*C_p),
            (S_1-x[3])*x[4]*H*L+E*A*S+P*L*E*S,
            g*H/(2*ro_0*f_0*L)*(alpha_T*(T_1-x[0])-alpha_S*(S_1-x[2]))-x[4],
            c*x[4]*(T_1-x[0])-x[5],
            c*x[4]*(S_1-x[2])-x[6]]

[T_0,T_2,S_0,S_2,V_1,T_eddy,S_eddy] = fsolve(main, [1,1,1,1,1,1,1])

In [4]:
[T_0,T_2,S_0,S_2,V_1,T_eddy,S_eddy]

[6.224002138509651,
 6.458278362452684,
 35.08894843551929,
 35.10804930078094,
 0.3501876723572814,
 0.005559574995615423,
 0.00023333333333333333]

In [ ]:
#### Two main equations #####

#Interior temperature [Celsius] -> without the effect of salinity
T_0 = T_1 - (mu/epsilon)*(M.sqrt(1+2*epsilon/mu)-1)*(T_1 - T_A)
Delta_T = (T_1-T_0)/(T_1-T_A)

#Interior salinity [g kg^-1]
S_0 = S_1 + 2*A*E*S*ro_0*f_0*L/(g*c*P*H**2*alpha_T*(T_1-T_0))
#Delta_S = 0.5*(Delta_T + M.sqrt(Delta_T**2 + gamma/epsilon)) #haline mode
#S_0 = S_1 - Delta_S*alpha_T*(T_1-T_A)/alpha_S


In [ ]:
#### Follow up computations ####

#Velocity in boundary current [m/s]
V_1 = g*H/(2*ro_0*f_0*L)*(alpha_T*(T_1-T_0) - alpha_S*(S_1-S_0))

#Outflowing salinity [g kg^-1]
S_out = S_1 + E*A*S_0/(V_1*H*L)

#Outflowing temperature [Celsius]
T_out = A*Gamma/(ro_0*C_p*V_1*H*L)*(T_0-T_A+P*L/A*(T_1-T_A))

#Densities -> not sure they are correct though..
ro_1 = ro_0*(1-alpha_T*(T_1-T_0) + alpha_S*(S_1-S_0))
ro_out = ro_0*(1-alpha_T*(T_out-T_0) + alpha_S*(S_out-S_0))

#Downwelling/Medidional overturning strength at the sill [m^2/s]
W = g*H**2*(ro_1 - ro_out)/(4*ro_0*f_0)

In [ ]:
T_0, S_0, V_1, S_out, T_out, W

In [ ]:
#Without salinity in the equation of state

def main(x): # function with four unkowns: T_0, T_2, S_0, S_2 = x[0],x[1],x[2],x[3]
    return [alpha_T*g/(2*ro_0*f_0)*(T_1-x[0])*H/L*c*(T_1-x[0])*P*H-A*Gamma*(x[0]-T_A)/(ro_0-T_A),                        
           
            alpha_T*g/(2*ro_0*f_0)*(T_1-x[0])*H/L*c*(S_1-x[2])*P*H+A*E*S,
           
            alpha_T*g/(2*ro_0*f_0)*((T_1-x[0])*H/L)*(T_1-x[1])*H*L+A*Gamma/(ro_0*C_p)*((1+P*L/A)*(T_1-T_A)-(T_1-x[0])),   
                
            alpha_T*g/(2*ro_0*f_0)*((T_1-x[0])*H/L)*(S_1-x[3])+E*A*S]          

In [ ]:
root = fsolve(main, [1,1,1,1])
root

In [ ]:
#Without salinity in the equation of state

def main(x):
    return[-x[0] + T_1 - (mu/epsilon)*(M.sqrt(1+2*epsilon/mu)-1)*(T_1 - T_A), 
           -x[1] + S_1 - 2*A*E*S*ro_0*f_0*L/(g*c*P*H**2*alpha_T*(T_1-x[0])),
           -x[2] + A*Gamma/(ro_0*C_p*g*H/(2*ro_0*f_0*L)*(alpha_T*(T_1-x[0]) - alpha_S*(S_1-x[1]))*H*L)*(x[0]-T_A+P*L/A*(T_1-T_A)),
           -x[3] + S_1 + E*A*S_0/(g*H/(2*ro_0*f_0*L)*(alpha_T*(T_1-x[0]) - alpha_S*(S_1-x[1]))*H*L)]

In [ ]:
root = fsolve(main, [1,1,1,1])
root

In [ ]:
#Including salinity in the equation of state

def main(x):
    return[ g*c*P*H**2*(alpha_T*(T_1-x[0])-alpha_S*(S_1-x[1]))*(T_1-x[0])/(2*ro_0*f_0*L)-A*Gamma*(x[0]-T_A)/(ro_0*C_p),
            g*c*P*H**2*(alpha_T*(T_1-x[0])-alpha_S*(S_1-x[1]))*(S_1-x[1])/(2*ro_0*f_0*L) + A*E*S,
            (T_1-x[2])*g*H/(2*ro_0*f_0*L)*(alpha_T*(T_1-x[0]) - alpha_S*(S_1-x[1]))*H*L - A*Gamma*(x[0]-T_A + P*L/A*(T_1-T_A))/(ro_0*C_p),
            (S_1-x[3])*g*H/(2*ro_0*f_0*L)*(alpha_T*(T_1-x[0]) - alpha_S*(S_1-x[1]))*H*L + E*A*S + P*L*E*S]

In [ ]:
root = fsolve(main, [1,1,1,1])
root